In [2]:
import gurobipy as gb
import networkx as nx

In [4]:
def addcut(edges):
    G = nx.Graph()
    G.add_nodes_from(V)
    for i, j in edges :
        G.add_edge(i, j)
        
    Components = nx.connected_components(G)
    if len(Components) == 1:
        return False
    for S in Components :
        model.addConstr(gb.quicksum(x[i,j] for i in S for j in S if j> i)<= len(S)-1)
    return True

In [5]:
def solve_tsp(V, c):
    model = gb.Model()
    x = {}
    for i in V :
        for j in V :
            if j > i :
                x[i, j] = model.addVar(ub = 1)
    model.update()
    for i in V:
        model.addConstr(gb.quicksum(x[j, i] for j in V if j< i)+gb.quicksum(x[i, j] for j in V if j > i) == 2)
    model.setObjective(gb.quicksum(c[i,j]*x[i,j] for i in V for j in V if j> i), gb.GRB.MINIMIZE)
    
    ESP = 1.e-6
    while True :
        model.optimize()
        edges = []
        for i,j in x :
            if x[i, j].X > ESP :
                edges.append((i,j))
        if addcut(edges) == False :
            if model.IsMIP :
                break
            for i,j in x :
                x[i,j].VType = 'B'
            model.update()
            
    return mode.ObjVal, edges